# Question 1 solution

1. Write a query which counts the amount of matches which took place in 2018 and had at least one premium user participating.

Solution a

```sql
SELECT game_id, count(*)
FROM (
      SELECT DISTINCT game_id, match_id
      FROM faceit
      WHERE membership = 'premium'
      -- AND YEAR(created_at) = 2018
      AND strftime('%Y', created_at) = 2018
      GROUP BY game_id, match_id
)
GROUP BY game_id
```

Solution b

```sql
SELECT game_id, COUNT(*)
FROM (
    SELECT DISTINCT game_id, match_id
    FROM faceit
    WHERE membership = 'premium'
    -- AND YEAR(created_at) = 2018
    AND strftime('%Y', created_at) = 2018
    GROUP BY game_id, match_id, membership
    HAVING COUNT(*) >= 1
)
GROUP BY game_id
```

2. Write a query which finds the list of all users who had at least one winning streak of 3 matches on the platform. A streak here is defined as achieving three or more consecutive wins in the same game. If a user won a match then his/her faction will be the same as the faction in the winner column.

```sql
WITH user_streaks AS (
  SELECT
    user_id,
    match_id,
    game_id,
    created_at,
    iif (faction=winner, true, false) AS result_win,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at) as user_order,
    ROW_NUMBER() OVER (PARTITION BY user_id, iif (faction=winner, true, false) ORDER BY created_at ASC) AS user_condition_order,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at) - ROW_NUMBER() OVER (PARTITION BY user_id, iif (faction=winner, true, false) ORDER BY created_at ASC) AS streak_id
  FROM faceit
)
SELECT DISTINCT user_id
FROM user_streaks
WHERE result_win = true
GROUP BY user_id, streak_id
HAVING count(*) >= 3
```